In [ ]:
import random
from typing import List, Dict, Any

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(False)

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
    {
        "movie": "Inception",
        "answer": """I will summarise the movie "Inception"
            - Director: Christopher Nolan
            - Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page
            - Budget: $160 million
            - Box Office Revenue: $836 million
            - Genre: Science Fiction, Thriller
            - Synopsis: A thief who enters the dreams of others to steal ideas leads a mission to plant an idea into the mind of a CEO.
            """,
    },
    {
        "movie": "The Matrix",
        "answer": """I will summarise the movie "The Matrix"
            - Director: Lana Wachowski, Lilly Wachowski
            - Main Cast: Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss
            - Budget: $63 million
            - Box Office Revenue: $465.3 million
            - Genre: Action, Science Fiction
            - Synopsis: A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.
            """,
    },
    {
        "movie": "Titanic",
        "answer": """
        I will sumarise the moveie "Titanic"
            - Director: James Cameron
            - Main Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane
            - Budget: $200 million
            - Box Office Revenue: $2.187 billion
            - Genre: Romance, Drama
            - Synopsis: A young aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.
        """,
    },
        {
        "movie": "The Dark Knight",
        "answer": """I will summarise the movie "The Dark Knight"
            - Director: Christopher Nolan
            - Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart
            - Budget: $185 million
            - Box Office Revenue: $1.005 billion
            - Genre: Action, Crime, Drama
            - Synopsis: Batman faces the Joker, a criminal mastermind wreaking havoc on Gotham City.
            """,
    },
]

# ─── RandomExampleSelector   ────────────────────────
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)
        
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

selector = RandomExampleSelector(examples)

# ─── example_prompt: {movie} 변수 사용 ─────────────────
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai",    "{answer}"),
    ]
)

# ─── few_shot_chat_message 템플릿 생성 (input_variables 지정) ───────
few_shot_chat_message = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=selector,
    input_variables=["movie"],
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
            "You are a helpful assistant that provides movie information.\n"
            "Given a movie title, summarise its director, main cast, budget, box office, genre, and a brief synopsis."
        ),
        few_shot_chat_message,  
        ("human", "What do you know about {movie}?"),
    ]
)
 
chain = final_prompt | chat
 
chain.invoke({"movie": "Interstellar"})


        I will sumarise the moveie "Interstellar"
            - Director: Christopher Nolan
            - Main Cast: Matthew McConaughey, Anne Hathaway, Jessica Chastain
            - Budget: $165 million
            - Box Office Revenue: $677.5 million
            - Genre: Science Fiction, Drama
            - Synopsis: In a future where Earth is becoming uninhabitable, a group of astronauts embarks on a dangerous mission through a wormhole near Saturn to find a new habitable planet for humanity.

AIMessageChunk(content='\n        I will sumarise the moveie "Interstellar"\n            - Director: Christopher Nolan\n            - Main Cast: Matthew McConaughey, Anne Hathaway, Jessica Chastain\n            - Budget: $165 million\n            - Box Office Revenue: $677.5 million\n            - Genre: Science Fiction, Drama\n            - Synopsis: In a future where Earth is becoming uninhabitable, a group of astronauts embarks on a dangerous mission through a wormhole near Saturn to find a new habitable planet for humanity.')